In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType
from pyspark.sql.functions import expr, array, col, explode, arrays_zip, when, first, avg
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from functools import reduce
from pyspark.sql import DataFrame

# warehouse_location points to the default location for managed databases and tables
warehouse = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
# Carregar tabelas
time_df = spark.table("database.time")
part_time_df = spark.table("database.gender")

# Lista de países e anos
paises = ["%Germany%", "%Switzerland%", "%Poland%", "%Hungary%"]
anos = ["2010", "2013"]

# Filtrando os DataFrames

def filter_by_multiple_conditions(df: DataFrame, paises: list, anos: list) -> DataFrame:
    conditions = reduce(lambda a, b: a | b, (col("nome_pais").like(p) for p in paises))
    df = df.filter(conditions)
    df = df.filter(col("ano").isin(anos))
    return df

time_df = filter_by_multiple_conditions(time_df, paises, anos)
part_time_df = filter_by_multiple_conditions(part_time_df, paises, anos)

time_df = time_df.drop("cod_indicador", "cod_pais")

part_time_df = part_time_df.drop("Country_code", "Country_name", "Indicator_name", "Indicator_code", "cod_indicador", "cod_pais")

# Filtrar pelos indicadores de emprego
part_time_df = part_time_df.filter(F.col("nome_indicador").like("%Part time employment%"))

# Adicionar a coluna "genero" com base no nome do indicador
part_time_df = part_time_df.withColumn("desagregacao",
    F.when(F.col("nome_indicador").like("% female%"), "female")
    .when(F.col("nome_indicador").like("% male%"), "male")
)
part_time_df = part_time_df.filter(F.col("desagregacao").isNotNull())

# Calcular a média dos valores por ano, gênero e indicador
part_time_df_media = part_time_df.groupBy("nome_pais", "ano", "desagregacao", "nome_indicador") \
    .agg(F.avg("valor").alias("valor"))

# Selecionar apenas uma linha por ano e gênero
window = Window.partitionBy("nome_pais", "ano", "desagregacao").orderBy(F.col("valor"))

part_time_df_final = part_time_df_media.withColumn("row_number", F.row_number().over(window)) \
    .filter(F.col("row_number") == 1).drop("row_number")

columns_order = ["nome_indicador", "nome_pais", "ano", "valor", "desagregacao"]

part_time_df_final = part_time_df_final.select(columns_order)

In [3]:
time_df = time_df.withColumn("valor", col("valor").cast("float"))
part_time_df_final = part_time_df_final.withColumn("valor", col("valor").cast("float"))

In [4]:
df_concatenado = part_time_df_final.union(time_df)

In [5]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

df_concatenado.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("hdfs://hdfs-nn:9000/Projeto/gold/TabelaPartTimeGold")

In [6]:
spark.sql(
    """
    SELECT * FROM database.PartTimeGold
    """
).show(500)

+-----------+--------------------+----+--------+------------+
|  nome_pais|      nome_indicador| ano|   valor|desagregacao|
+-----------+--------------------+----+--------+------------+
|    Germany|Proportion of tim...|2013|    16.4|      female|
|    Hungary|Proportion of tim...|2010|17.27674|      female|
|     Poland|Proportion of tim...|2013| 18.2625|      female|
|Switzerland|Proportion of tim...|2013| 16.5375|      female|
|Switzerland|Proportion of tim...|2010|16.74167|      female|
|    Germany|Proportion of tim...|2013|    10.4|        male|
|    Hungary|Proportion of tim...|2010|  7.9796|        male|
|     Poland|Proportion of tim...|2013| 9.92917|        male|
|Switzerland|Proportion of tim...|2013| 10.3875|        male|
|Switzerland|Proportion of tim...|2010|   9.775|        male|
|    Germany|Part time employm...|2010|   55.47|      female|
|    Germany|Part time employm...|2010|   22.98|        male|
|    Germany|Part time employm...|2013|   57.78|      female|
|    Ger

In [7]:
spark.stop()